### Preliminary setup

In [ ]:
# For data transformation
import pandas as pd            
# For statistical analysis
import statistics as stat
# For ASN lookup
import pyasn
asndb = pyasn.pyasn('ipasn_20140513.dat')

In [ ]:
# input csv
input_file = "./benign_flow/yt_test/test10.csv"
df = pd.read_csv(input_file)

column_names = ["DURATION", "SRC_IP", "DST_IP", "SRC_PORT", "DST_PORT", "QUIC_VERSION", "BYTES", "PROTOCOL"]
df.columns = column_names

# delete protocol column
df = df.drop("PROTOCOL", axis=1)

df["BYTES"] = pd.to_numeric(df["BYTES"], errors='coerce').fillna(0)
#df["DURATION"] = df["DURATION"] - df["DURATION"].iloc[0]

In [ ]:
# set source IP, web service category, and label
ipsrc = "10.10.3.10"
cat = "Streaming"
label = "0"

# determine the destination port and QUIC version


# initialize the interpacket variables
flow = []
arr_fwd_bytes = []
arr_rev_bytes = []
arr_fwd_iat = []
arr_rev_iat = []
arr_port = []
arr_asn = []
arr_ver = []
fwd_pkt = 0
fwd_bytes = 0.0
rev_pkt = 0
rev_bytes = 0.0
init_dur = 0.0
dur = 0.0
fwd_dur = 0.0
rev_dur = 0.0
ctr = 0

for index, value in df['SRC_IP'].iteritems():
    if value == ipsrc and ctr < 29:
        if index == 0:
            init_dur = df['DURATION'][index]
        else:
            fwd_dur += df['DURATION'][index] - init_dur
            arr_fwd_iat.append(df['DURATION'][index] - init_dur)
        ctr += 1
        arr_port.append(df['DST_PORT'][index])
        arr_asn.append(asndb.lookup(df['DST_IP'][index])[0])
        arr_ver.append(df['QUIC_VERSION'][index])
        fwd_pkt += 1
        fwd_bytes += df['BYTES'][index]
        arr_fwd_bytes.append(df['BYTES'][index])
    elif value != ipsrc and ctr < 29:
        if index == 0:
            init_dur = df['DURATION'][index]
        else:
            rev_dur += df['DURATION'][index] - init_dur
            arr_rev_iat.append(df['DURATION'][index] - init_dur)
        ctr += 1
        arr_port.append(df['SRC_PORT'][index])
        arr_asn.append(asndb.lookup(df['SRC_IP'][index])[0])
        arr_ver.append(df['QUIC_VERSION'][index])
        rev_pkt += 1
        rev_bytes += df['BYTES'][index]
        arr_rev_bytes.append(df['BYTES'][index])
    elif ctr == 29 or index == len(df['SRC_IP']) - 1:
        if value == ipsrc:
            fwd_dur += df['DURATION'][index] - init_dur
            arr_fwd_iat.append(df['DURATION'][index] - init_dur)
            arr_port.append(df['DST_PORT'][index])
            arr_asn.append(asndb.lookup(df['DST_IP'][index])[0])
            arr_ver.append(df['QUIC_VERSION'][index])
            fwd_pkt += 1
            fwd_bytes += df['BYTES'][index]
            arr_fwd_bytes.append(df['BYTES'][index])
        else:
            rev_dur += df['DURATION'][index] - init_dur
            arr_rev_iat.append(df['DURATION'][index] - init_dur)
            arr_port.append(df['SRC_PORT'][index])
            arr_asn.append(asndb.lookup(df['SRC_IP'][index])[0])
            arr_ver.append(df['QUIC_VERSION'][index])
            rev_pkt += 1
            rev_bytes += df['BYTES'][index]
            arr_rev_bytes.append(df['BYTES'][index])

        dst_port = stat.mode(arr_port)
        dst_asn = stat.mode([x for x in arr_asn if isinstance(x, int)])
        quic_ver = stat.mode(arr_ver)
        
        new_quic_ver = [x.strip() for x in quic_ver.split(',')]
        # Convert each hex value to integer
        temp = []
        for hex_val in new_quic_ver:
            # Verify format
            if not hex_val.startswith('0x'):
                continue
            # Convert to integer
            num = int(hex_val, 16)
            temp.append(num)
        quic_ver = max(temp)
        
        dur -= df['DURATION'][index]
        trips = ctr + 1
        ratio = fwd_pkt / rev_pkt
        tot_pkt = fwd_pkt + rev_pkt
        tot_bytes = fwd_bytes + rev_bytes
        flow_pkt = tot_pkt / dur
        flow_bytes = tot_bytes / dur
        max_bytes = max(fwd_bytes + rev_bytes)
        min_bytes = min(fwd_bytes + rev_bytes)
        ave_bytes = stat.mean(fwd_bytes + rev_bytes)
        std_bytes = stat.stdev(fwd_bytes + rev_bytes)
        var_bytes = stat.variance(fwd_bytes + rev_bytes)
        max_fwd_bytes = max(arr_fwd_bytes)
        min_fwd_bytes = min(arr_fwd_bytes)
        ave_fwd_bytes = stat.mean(arr_fwd_bytes)
        std_fwd_bytes = stat.stdev(arr_fwd_bytes)
        var_fwd_bytes = stat.variance(arr_fwd_bytes)
        max_rev_bytes = max(arr_rev_bytes)
        min_rev_bytes = min(arr_rev_bytes)
        ave_rev_bytes = stat.mean(arr_rev_bytes)
        std_rev_bytes = stat.stdev(arr_rev_bytes)
        var_rev_bytes = stat.variance(arr_rev_bytes)
        max_iat = max(arr_fwd_iat + arr_rev_iat)
        min_iat = min(arr_fwd_iat + arr_rev_iat)
        ave_iat = stat.mean(arr_fwd_iat + arr_rev_iat)
        std_iat = stat.stdev(arr_fwd_iat + arr_rev_iat)
        var_iat = stat.variance(arr_fwd_iat + arr_rev_iat)
        max_fwd_iat = max(arr_fwd_iat)
        min_fwd_iat = min(arr_fwd_iat)
        ave_fwd_iat = stat.mean(arr_fwd_iat)
        std_fwd_iat = stat.stdev(arr_fwd_iat)
        var_fwd_iat = stat.variance(arr_fwd_iat)
        max_rev_iat = max(arr_rev_iat)
        min_rev_iat = min(arr_rev_iat)
        ave_rev_iat = stat.mean(arr_rev_iat)
        std_rev_iat = stat.stdev(arr_rev_iat)
        var_rev_iat = stat.variance(arr_rev_iat)
        ctr = 0
        
        flow.append([dst_port, dst_asn, quic_ver, dur, cat, trips, ratio, flow_pkt, flow_bytes, tot_pkt, tot_bytes, max_bytes, min_bytes, ave_bytes, std_bytes, var_bytes, fwd_pkt, fwd_bytes, max_fwd_bytes, min_fwd_bytes, ave_fwd_bytes, std_fwd_bytes, var_fwd_bytes, rev_pkt, rev_bytes, max_rev_bytes, min_rev_bytes, ave_rev_bytes, std_rev_bytes, var_rev_bytes, max_iat, min_iat, ave_iat, std_iat, var_iat, fwd_dur, max_fwd_iat, min_fwd_iat, ave_fwd_iat, std_fwd_iat, var_fwd_iat, rev_dur, max_rev_iat, min_rev_iat, ave_rev_iat, std_rev_iat, var_rev_iat, label])

        arr_fwd_bytes = []
        arr_rev_bytes = []
        arr_fwd_iat = []
        arr_rev_iat = []
        arr_port = []
        arr_asn = []
        arr_ver = []
        fwd_pkt = 0
        fwd_bytes = 0.0
        rev_pkt = 0
        rev_bytes = 0.0
        init_dur = 0.0
        dur = 0.0
        fwd_dur = 0.0
        rev_dur = 0.0
        ctr = 0

  

In [ ]:
for item in flow:
    print(item)

In [5]:
# Read the existing CSV file
file_path = "./benign_flow/benign.csv"
df = pd.read_csv(file_path)

# Append each item in the flow list as a new row in the DataFrame
for item in flow:
    df.loc[len(df)] = item

# Write the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

In [6]:
# Save the filtered DataFrame to a new CSV file
output_file = "./benign_flow/filtered_output.csv"
df.to_csv(output_file, index=False)

### archive

In [ ]:
# source IP
ipsrc = df["SRC_IP"].iloc[0]
portsrc = df["SRC_PORT"].iloc[0]
cat = "Streaming"

flow = []

df["true_dest"] = df.apply(lambda row: row["DST_IP"] if row["SRC_IP"] == ipsrc else row["SRC_IP"], axis=1)
df["group"] = (df["true_dest"] != df["true_dest"].shift()).cumsum()

for group, group_df in df.groupby("group"):
    num_subgroups = (len(group_df) + 29) // 30
    subgroups = [group_df.iloc[i * 30:(i+1) * 30] for i in range(num_subgroups)]
    
    # print(f"Group {group}:")
    # print(group_df)

    for  subgroup in subgroups:
        ppi_dir = []
        ipdst = subgroup["true_dest"].iloc[0]  # The unique normalized destination for this subgroup
        portdst = subgroup["DST_PORT"].iloc[0] 

        ppi_time = [0]
        ppi_size = [int(subgroup["BYTES"].iloc[0])]
        for i in range(1, len(subgroup)):
            # Calculate the time difference between consecutive packets
            duration = int((subgroup["DURATION"].iloc[i] - subgroup["DURATION"].iloc[i - 1]) * 1000)
            ppi_time.append(duration)
            ppi_size.append(int(subgroup["BYTES"].iloc[i]))



        for _, row in subgroup.iterrows():
            if row["SRC_IP"] == ipsrc and row["DST_IP"] == ipdst:
                ppi_dir.append(1)
            elif row["SRC_IP"] == ipdst  and row["DST_IP"] == ipsrc:
                ppi_dir.append(-1)
            else:
                ppi_dir.append(0)

            bytes_fromsrc = int(subgroup.loc[(subgroup["SRC_IP"] == ipsrc) & (subgroup["DST_IP"] == ipdst),"BYTES"].sum())
            bytes_rev = int(subgroup.loc[(subgroup["SRC_IP"] == ipdst) & (subgroup["DST_IP"] == ipsrc),"BYTES"].sum())
            packets = int(subgroup.loc[(subgroup["SRC_IP"] == ipsrc) & (subgroup["DST_IP"] == ipdst),"BYTES"].count())
            packets_rev = int(subgroup.loc[(subgroup["SRC_IP"] == ipdst) & (subgroup["DST_IP"] == ipsrc),"BYTES"].count())
            ppi_len = len(ppi_dir)

            ppi_rtt = 0
            in_group = False

            # Iterate through the list
            for value in ppi_dir:
                if value == -1:  # Start or continue a group of -1's
                    if not in_group:
                        in_group = True  # Beginning of a group
                elif value == 1:  # Start or continue a group of 1's
                    if not in_group:
                        in_group = True  # Beginning of a group
                    elif in_group:
                        ppi_rtt += 1
                        in_group = False  # Reset for the next group

            # Handle the case where the list ends with a valid pair
            if in_group:
                ppi_rtt += 1

        # print("Subgroup:")
        # print(subgroup)

        dur = round(subgroup["DURATION"].max() - subgroup["DURATION"].min(), ndigits=6)

        #flow.append([ipsrc, ipdst, asndb.lookup(ipdst)[0], portsrc, portdst, 1, sni, dur, bytes_fromsrc, bytes_rev, packets, packets_rev, ppi_len, ppi_rtt, cat, [ppi_time, ppi_dir, ppi_size]])
        flow.append([portdst, asndb.lookup(ipdst)[0], portsrc, portdst, 1, sni, dur, bytes_fromsrc, bytes_rev, packets, packets_rev, ppi_len, ppi_rtt, cat, [ppi_time, ppi_dir, ppi_size]])
        